# Marketing Leads Conversion - Missing Data
- Identify Missing Values
- Impute Missing Values
    - Continuous
    - Categorical

## 1/ Import Libraries

In [1]:
# pandas, numpy
import pandas as pd
import numpy as np

## 2/ Import Data
- t1_leads
- t6_emi
- test

**Extra:** Join other tables to build a more comprehensive training data

In [2]:
# Train
train = pd.read_csv('data/train.csv', low_memory=False)

#Test Data - Final Predictions
test = pd.read_csv('data/test.csv', low_memory=False)

### Check first 5 rows

In [3]:
train.head()

ID  Gender       DOB Lead_Creation_Date City_Code  \
0  APPC90493171225  Female  23/07/79           15/07/16    C10001   
1  APPD40611263344    Male  07/12/86           04/07/16    C10003   
2  APPE70289249423    Male  10/12/82           19/07/16    C10125   
3  APPF80273865537    Male  30/01/89           09/07/16    C10477   
4  APPG60994436641    Male  19/04/85           20/07/16    C10002   

  City_Category Employer_Code Employer_Category1  Employer_Category2  \
0             A    COM0044082                  A                 4.0   
1             A    COM0000002                  C                 1.0   
2             C    COM0005267                  C                 4.0   
3             C    COM0004143                  A                 4.0   
4             A    COM0001781                  A                 4.0   

   Monthly_Income  ... Contacted Source Source_Category Existing_EMI  \
0          2000.0  ...         N   S122               G          0.0   
1          3500.0  ...         Y   S122               G          0.0   
2          2250.0  ...         Y   S143               B          0.0   
3          3500.0  ...         Y   S143               B          0.0   
4         10000.0  ...         Y   S134               B       2500.0   

  Loan_Amount  Loan_Period  Interest_Rate    EMI  Var1  Approved  
0         NaN          NaN            NaN    NaN     0         0  
1     20000.0          2.0          13.25  953.0    10         0  
2     45000.0          4.0            NaN    NaN     0         0  
3     92000.0          5.0            NaN    NaN     7         0  
4     50000.0          2.0            NaN    NaN    10         0  

[5 rows x 22 columns]

In [4]:
#test
test.head()

ID  Gender       DOB Lead_Creation_Date City_Code  \
0  APPA70109647212    Male  03/06/88           05/07/16    C10028   
1  APPB10687939341    Male  13/07/81           01/07/16    C10003   
2  APPC80449411414  Female  19/11/90           01/07/16    C10009   
3  APPD30665094501  Female  15/10/92           01/07/16    C10005   
4  APPE80379821637    Male  21/09/88           01/07/16    C10005   

  City_Category Employer_Code Employer_Category1  Employer_Category2  \
0             C    COM0002222                  A                 4.0   
1             A    COM0001784                  C                 4.0   
2             B    COM0045260                  B                 4.0   
3             A    COM0000085                  A                 3.0   
4             A    COM0006422                  A                 4.0   

   Monthly_Income  ... Primary_Bank_Type Contacted Source Source_Category  \
0          2150.0  ...                 P         Y   S122               B   
1          4200.0  ...                 P         Y   S133               C   
2          1000.0  ...                 P         N   S133               B   
3          1465.0  ...                 P         N   S133               C   
4          2340.0  ...                 P         Y   S143               B   

  Existing_EMI  Loan_Amount  Loan_Period  Interest_Rate     EMI  Var1  
0          0.0      10000.0          3.0           20.0   372.0     4  
1          0.0      69000.0          5.0           24.0  1985.0     7  
2          0.0          NaN          NaN            NaN     NaN     0  
3          0.0          NaN          NaN            NaN     NaN     0  
4        500.0      10000.0          2.0            NaN     NaN     0  

[5 rows x 21 columns]

## 3/ Convert String to Date Format

### Convert: Lead_Creation_Date

In [5]:
# Train
train['Lead_Creation_Date'] = pd.to_datetime(train['Lead_Creation_Date'], infer_datetime_format = True)
train['DOB'] = pd.to_datetime(train['DOB'], infer_datetime_format = True)

# Test (need to do the same for Test too)
test['Lead_Creation_Date'] = pd.to_datetime(test['Lead_Creation_Date'], infer_datetime_format = True)
test['DOB'] = pd.to_datetime(test['DOB'], infer_datetime_format = True)

## 5/ Identify Missing Values

### Option 2: isnull + sum

In [8]:
train.isnull().sum()

ID                                         0
Gender                                     0
DOB                                       15
Lead_Creation_Date                         0
City_Code                                814
City_Category                            814
Employer_Code                           4018
Employer_Category1                      4018
Employer_Category2                      4298
Monthly_Income                             0
Customer_Existing_Primary_Bank_Code     9391
Primary_Bank_Type                       9391
Contacted                                  0
Source                                     0
Source_Category                            0
Existing_EMI                              51
Loan_Amount                            27709
Loan_Period                            27709
Interest_Rate                          47437
EMI                                    47437
Var1                                       0
Approved                                   0
dtype: int

## 6/ Impute Missing Values

### 1/ Continuous - Indentify and Impute

In [10]:
numerical_features = [feature for feature in train.columns if train[feature].dtypes in ['float64','int64'] and feature !='ID' and feature !='Approved']
numerical_features

['Employer_Category2',
 'Monthly_Income',
 'Existing_EMI',
 'Loan_Amount',
 'Loan_Period',
 'Interest_Rate',
 'EMI',
 'Var1']

In [11]:
# Check Missing
train[numerical_features].isnull().sum()

Employer_Category2     4298
Monthly_Income            0
Existing_EMI             51
Loan_Amount           27709
Loan_Period           27709
Interest_Rate         47437
EMI                   47437
Var1                      0
dtype: int64

#### Impute Numerical with SimpleImputer
1. Extract Numerical Features from train
2. Import Package - Impute
3. Generate Instance
4. Calculate the Median
5. Impute the features
6. Copy it back to the Train 

In [12]:
# import package
from sklearn.impute import SimpleImputer

In [16]:
# Create Instance
imp_num = SimpleImputer(strategy = 'median')

In [17]:
# Fit - Calculate Median for all columns
imp_num.fit(train[numerical_features])

SimpleImputer(strategy='median')

In [34]:
# Transform - Impute Missing Values with Median
train[numerical_features] = imp_num.transform(train[numerical_features])
test[numerical_features] = imp_num.transform(test[numerical_features])

#### Check Again

In [21]:
train.isnull().sum()

ID                                        0
Gender                                    0
DOB                                      15
Lead_Creation_Date                        0
City_Code                               814
City_Category                           814
Employer_Code                          4018
Employer_Category1                     4018
Employer_Category2                        0
Monthly_Income                            0
Customer_Existing_Primary_Bank_Code    9391
Primary_Bank_Type                      9391
Contacted                                 0
Source                                    0
Source_Category                           0
Existing_EMI                              0
Loan_Amount                               0
Loan_Period                               0
Interest_Rate                             0
EMI                                       0
Var1                                      0
Approved                                  0
dtype: int64

### 3/ Categorical - Identify and Impute

In [22]:
categorical_features = [feature for feature in train.columns if train[feature].dtypes=='O' and feature !='ID' and feature !='Approved']
categorical_features

['Gender',
 'City_Code',
 'City_Category',
 'Employer_Code',
 'Employer_Category1',
 'Customer_Existing_Primary_Bank_Code',
 'Primary_Bank_Type',
 'Contacted',
 'Source',
 'Source_Category']

#### Impute Categorical with SimpleImputer

In [37]:
# import package
from sklearn.impute import SimpleImputer

In [23]:
# Create Instance
imp_cat = SimpleImputer(strategy = 'constant', fill_value = 'missing')

In [25]:
# Fit
imp_cat.fit(train[categorical_features])

SimpleImputer(fill_value='missing', strategy='constant')

In [35]:
# Transform
train[categorical_features] = imp_cat.transform(train[categorical_features])
test[categorical_features] = imp_cat.transform(test[categorical_features])

# Back at 3:50 

#### Check Again

In [27]:
train.isnull().sum()

ID                                      0
Gender                                  0
DOB                                    15
Lead_Creation_Date                      0
City_Code                               0
City_Category                           0
Employer_Code                           0
Employer_Category1                      0
Employer_Category2                      0
Monthly_Income                          0
Customer_Existing_Primary_Bank_Code     0
Primary_Bank_Type                       0
Contacted                               0
Source                                  0
Source_Category                         0
Existing_EMI                            0
Loan_Amount                             0
Loan_Period                             0
Interest_Rate                           0
EMI                                     0
Var1                                    0
Approved                                0
dtype: int64

## 7/ Build Model

## Prepare Data

In [28]:
#create X and y datasets for splitting 
X = train.drop(['ID', 'Approved','DOB', 'Lead_Creation_Date', 'City_Code', 'Employer_Code', 'Customer_Existing_Primary_Bank_Code', 'Source'], axis=1)
y = train['Approved']

## Create Dummy

In [29]:
X = pd.get_dummies(X, drop_first=True)

In [30]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69713 entries, 0 to 69712
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Employer_Category2          69713 non-null  float64
 1   Monthly_Income              69713 non-null  float64
 2   Existing_EMI                69713 non-null  float64
 3   Loan_Amount                 69713 non-null  float64
 4   Loan_Period                 69713 non-null  float64
 5   Interest_Rate               69713 non-null  float64
 6   EMI                         69713 non-null  float64
 7   Var1                        69713 non-null  float64
 8   Gender_Male                 69713 non-null  uint8  
 9   City_Category_B             69713 non-null  uint8  
 10  City_Category_C             69713 non-null  uint8  
 11  City_Category_missing       69713 non-null  uint8  
 12  Employer_Category1_B        69713 non-null  uint8  
 13  Employer_Category1_C        697

## Import LR Package and create Instance

In [31]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter = 1000)

## Build Model

In [32]:
lr.fit(X,y)

LogisticRegression(max_iter=1000)

### Check Score

In [33]:
lr.score(X, y)

0.9852968599830735